In [1]:
from minio import Minio
import pandas as pd
import numpy as np
from io import BytesIO

client = Minio("0.0.0.0:9000",
    access_key="5MpW7tf4J1U4KKvzLm7X",
    secret_key="bOpLlAAeSqJgfZBvLZFNRZA9o1eP5Mc7l7ea8R4V",
    secure=False,
)

# result = client.put_object(
#     "datasets", "test.csv", data=csv_buffer, length=len(csv_bytes), content_type="application/csv"
# )
dataset_list = ["energy_dataset.csv", "weather_features.csv"]

def fetch_from_minio(filename, bucket_name):

    response = client.get_object(bucket_name, filename)
    
    try:
        file_content = BytesIO(response.read())
        df = pd.read_csv(file_content, index_col=False)
    finally:
        response.close()
        response.release_conn()
    return df

energy_df, weather_df = [fetch_from_minio(filename, "datasets") for filename in dataset_list]

In [ ]:
# Preprocess the file
df_list = [energy_df, weather_df]

for i, df in enumerate(df_list): #this for is only 2 items

    # Drop the columns that only contain Nones and/or NaNs and all 0s
    for col in df:
        if (
            df[col].isnull().all() or 
            (df[col].dtype != "datetime64[ns, UTC]" and df[col].sum() == 0)
        ):
            df = df.drop(columns=[col], axis=1)
    
    # Set the time to datetime type and set as index
    if "dt_iso" in df.columns:
        df['time'] = pd.to_datetime(df['dt_iso'], utc=True, infer_datetime_format=True)
        df = df.drop(['dt_iso'], axis=1) #not getting dropped here
        df = df.set_index('time')
    elif "time" in df.columns:
        df['time'] = pd.to_datetime(df['time'], utc=True, infer_datetime_format=True)
        df = df.set_index('time')

    #check for duplicates

    duplicate_row_count = df.duplicated(keep='first').sum()
    if duplicate_row_count > 0:
        df = df.reset_index().drop_duplicates(subset=['time', 'city_name'],
                                                keep='first').set_index('time')

    #convert any ints to floats
    cols = df.select_dtypes(include=[np.int64]).columns
    for col in cols:
        df[col] = df[col].values.astype(np.float64)
    
    #drop any textual columns
    df = df.drop(columns=[col for col in df.columns if df[col].dtypes == 'object' and col != "city_name"])

    #remove extreme pressure outliers for weather df
    if "pressure" in df.columns:
        df.loc[df.pressure > 1051, 'pressure'] = np.nan
        df.loc[df.pressure < 931, 'pressure'] = np.nan
    
    if "wind_speed" in df.columns:
        df.loc[df.wind_speed > 50, 'wind_speed'] = np.nan

    #interpolate any null values
    null_value_count = df.isnull().values.sum()
    if null_value_count > 0:
        df.interpolate(method='linear', limit_direction='forward', inplace=True, axis=0)

    df_list[i] = df

# seperate weather dfs
weather_df = df_list[1]
energy_df = df_list[0]
if "rain_3h" in weather_df.columns:
    weather_df = weather_df.drop(['rain_3h'], axis=1) #drop unreliable feature
weather_df_1, weather_df_2, weather_df_3, weather_df_4, weather_df_5 = [x for _, x in weather_df.groupby('city_name')]
weather_dfs = [weather_df_1, weather_df_2, weather_df_3, weather_df_4, weather_df_5]

#merge into one df
df_final = energy_df

for df in weather_dfs:
    city = df['city_name'].unique()
    city_str = str(city).replace("'", "").replace('[', '').replace(']', '').replace(' ', '')
    df = df.add_suffix('_{}'.format(city_str))
    df_final = df_final.merge(df, on=['time'], how='outer')
    df_final = df_final.drop('city_name_{}'.format(city_str), axis=1)

#save preprocessed df to minio
save_df_as_csv_to_minio(df_final, "merged_preprocessed.csv", idx=True)

/var/folders/v7/bdkd1w4x4lb24ppj_z8h56fh0000gn/T/ipykernel_98888/497808758.py:20: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['time'] = pd.to_datetime(df['time'], utc=True, infer_datetime_format=True)
/var/folders/v7/bdkd1w4x4lb24ppj_z8h56fh0000gn/T/ipykernel_98888/497808758.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['time'] = pd.to_datetime(df['dt_iso'], utc=True, infer_datetime_format=True)


In [27]:
weather_df

,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_3h,clouds_all,weather_id
time,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,Valencia,270.475,270.475,270.475,1001.0,77.0,1.0,62.0,0.0,0.0,0.0,800.0
2015-01-01 00:00:00+00:00,Valencia,270.475,270.475,270.475,1001.0,77.0,1.0,62.0,0.0,0.0,0.0,800.0
2015-01-01 01:00:00+00:00,Valencia,269.686,269.686,269.686,1002.0,78.0,0.0,23.0,0.0,0.0,0.0,800.0
2015-01-01 02:00:00+00:00,Valencia,269.686,269.686,269.686,1002.0,78.0,0.0,23.0,0.0,0.0,0.0,800.0
2015-01-01 03:00:00+00:00,Valencia,269.686,269.686,269.686,1002.0,78.0,0.0,23.0,0.0,0.0,0.0,800.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,Seville,287.760,287.150,288.150,1028.0,54.0,3.0,30.0,0.0,0.0,0.0,800.0
2018-12-31 19:00:00+00:00,Seville,285.760,285.150,286.150,1029.0,62.0,3.0,30.0,0.0,0.0,0.0,800.0
2018-12-31 20:00:00+00:00,Seville,285.150,285.150,285.150,1028.0,58.0,4.0,50.0,0.0,0.0,0.0,800.0


In [14]:
energy_df

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,...,277.0,0.0,3113.0,96.0,NaN,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-31 20:00:00+01:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,...,280.0,0.0,3288.0,51.0,NaN,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-31 21:00:00+01:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,...,286.0,0.0,3503.0,36.0,NaN,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-31 22:00:00+01:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,0.0,...,287.0,0.0,3586.0,29.0,NaN,3273.0,25450.0,25801.0,63.93,69.89


In [25]:
weather_df

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,time
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0,800,clear,sky is clear,01n,2014-12-31 23:00:00+00:00
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0,800,clear,sky is clear,01n,2015-01-01 00:00:00+00:00
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0,800,clear,sky is clear,01n,2015-01-01 01:00:00+00:00
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0,800,clear,sky is clear,01n,2015-01-01 02:00:00+00:00
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0,800,clear,sky is clear,01n,2015-01-01 03:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178391,2018-12-31 19:00:00+01:00,Seville,287.760,287.150,288.150,1028,54,3,30,0.0,0.0,0,800,clear,sky is clear,01n,2018-12-31 18:00:00+00:00
178392,2018-12-31 20:00:00+01:00,Seville,285.760,285.150,286.150,1029,62,3,30,0.0,0.0,0,800,clear,sky is clear,01n,2018-12-31 19:00:00+00:00
178393,2018-12-31 21:00:00+01:00,Seville,285.150,285.150,285.150,1028,58,4,50,0.0,0.0,0,800,clear,sky is clear,01n,2018-12-31 20:00:00+00:00
178394,2018-12-31 22:00:00+01:00,Seville,284.150,284.150,284.150,1029,57,4,60,0.0,0.0,0,800,clear,sky is clear,01n,2018-12-31 21:00:00+00:00


In [28]:
df_final = energy_df

for df in weather_dfs:
    city = df['city_name'].unique()
    city_str = str(city).replace("'", "").replace('[', '').replace(']', '').replace(' ', '')
    df = df.add_suffix('_{}'.format(city_str))
    df_final = df_final.merge(df, on=['time'], how='outer')
    df_final = df_final.drop('city_name_{}'.format(city_str), axis=1)

In [29]:
df_final.shape

(35064, 75)

In [30]:
35064 * 75

2629800

In [31]:
correlations = df_final.corr(method='pearson')
print(correlations['price actual'].sort_values(ascending=False).to_string())

price actual                                   1.000000
price day ahead                                0.732155
generation fossil hard coal                    0.465637
generation fossil gas                          0.461452
total load forecast                            0.435864
total load actual                              0.435253
generation fossil brown coal/lignite           0.363993
generation fossil oil                          0.285050
generation other renewable                     0.255551
pressure_Barcelona                             0.249177
pressure_Bilbao                                0.194063
generation waste                               0.168710
generation biomass                             0.142671
temp_min_Valencia                              0.133141
pressure_Valencia                              0.109812
temp_min_Barcelona                             0.103726
forecast solar day ahead                       0.101402
generation other                               0

In [33]:
highly_correlated = abs(correlations[correlations > 0.75])
print(highly_correlated[highly_correlated < 1.0].stack().to_string())

generation fossil brown coal/lignite  generation fossil hard coal             0.768831
generation fossil hard coal           generation fossil brown coal/lignite    0.768831
generation solar                      forecast solar day ahead                0.993244
generation wind onshore               forecast wind onshore day ahead         0.994457
forecast solar day ahead              generation solar                        0.993244
forecast wind onshore day ahead       generation wind onshore                 0.994457
total load forecast                   total load actual                       0.995096
total load actual                     total load forecast                     0.995096
temp_Barcelona                        temp_min_Barcelona                      0.970264
                                      temp_max_Barcelona                      0.976904
                                      temp_Bilbao                             0.866727
                                      temp_